# Visualización de Grafos de Conocimiento (Wikidata-only)

Este notebook carga un grafo RDF (`.ttl`) generado por tu pipeline y lo **plotea** con `networkx + matplotlib`.

## 1) Requisitos

In [2]:
# Ejecuta esta celda si te falta alguna librería.
# (Si ya las tienes, puedes omitirla.)
try:
    import rdflib, networkx, matplotlib
except Exception:
    %pip install rdflib networkx matplotlib


  Using cached matplotlib-3.10.7-cp312-cp312-macosx_11_0_arm64.whl.metadata (11 kB)
  Using cached contourpy-1.3.3-cp312-cp312-macosx_11_0_arm64.whl.metadata (5.5 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached fonttools-4.60.1-cp312-cp312-macosx_10_13_universal2.whl.metadata (112 kB)
  Using cached kiwisolver-1.4.9-cp312-cp312-macosx_11_0_arm64.whl.metadata (6.3 kB)
  Using cached numpy-2.3.4-cp312-cp312-macosx_14_0_arm64.whl.metadata (62 kB)
  Using cached pillow-12.0.0-cp312-cp312-macosx_11_0_arm64.whl.metadata (8.8 kB)
Using cached matplotlib-3.10.7-cp312-cp312-macosx_11_0_arm64.whl (8.1 MB)
Using cached contourpy-1.3.3-cp312-cp312-macosx_11_0_arm64.whl (273 kB)
Using cached cycler-0.12.1-py3-none-any.whl (8.3 kB)
Using cached fonttools-4.60.1-cp312-cp312-macosx_10_13_universal2.whl (2.8 MB)
Using cached kiwisolver-1.4.9-cp312-cp312-macosx_11_0_arm64.whl (64 kB)
Using cached numpy-2.3.4-cp312-cp312-macosx_14_0_arm64.whl (5.1 MB)
Using cached pillo

## 2) Configurar rutas

In [6]:
from pathlib import Path

REPO_ROOT = Path.cwd().resolve().parents[3]
GRAPHS_DIR = REPO_ROOT / "src" / "data" / "graphs" / "full"
assert GRAPHS_DIR.exists(), f"No encuentro {GRAPHS_DIR}. ¿Estás ejecutando dentro del repositorio?"

ttl_files = sorted(GRAPHS_DIR.glob("*.ttl"))
print(f"Se encontraron {len(ttl_files)} grafos en {GRAPHS_DIR}")
for p in ttl_files[:10]:
    print(" -", p.name)
TTL_FILE = ttl_files[0] if ttl_files else None
print("Usando:", TTL_FILE)


NameError: name '__file__' is not defined

## 3) Función de ploteo

In [ ]:
import matplotlib.pyplot as plt
import networkx as nx
from rdflib import Graph, URIRef, Namespace

WDT = Namespace("http://www.wikidata.org/prop/direct/")

def plot_graph_degree1(ttl_path, max_edges=40, figsize=(12, 9)):
    g = Graph()
    g.parse(ttl_path, format="turtle")
    G = nx.DiGraph()
    for s, p, o in g.triples((None, None, None)):
        if isinstance(s, URIRef) and isinstance(o, URIRef) and str(p).startswith(str(WDT)):
            G.add_edge(str(s), str(o), label=str(p))
    if G.number_of_edges() == 0:
        raise ValueError("No se encontraron aristas 'wdt:' en el grafo.")
    edges = list(G.edges)[:max_edges]
    H = G.edge_subgraph(edges).copy()
    pos = nx.spring_layout(H, k=0.6, seed=42)
    plt.figure(figsize=figsize)
    nx.draw(H, pos, with_labels=False, node_color="#a0cbe2", edge_color="#333333", node_size=500, arrowsize=16)
    node_labels = {n: n.split("/")[-1] for n in H.nodes()}
    nx.draw_networkx_labels(H, pos, node_labels, font_size=9)
    edge_labels = {(u, v): d.get("label", "").split("/")[-1] for u, v, d in H.edges(data=True)}
    nx.draw_networkx_edge_labels(H, pos, edge_labels=edge_labels, font_size=8)
    plt.title(f"Grafo RDF (nodos={H.number_of_nodes()}, aristas={H.number_of_edges()})")
    plt.axis("off")
    plt.show()


## 4) Plotear el grafo

In [ ]:
if TTL_FILE is None:
    raise SystemExit("No se encontró ningún TTL en graphs/full/. Genera grafos con el pipeline y reintenta.")
plot_graph_degree1(TTL_FILE, max_edges=40)
